In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#!unzip "/content/gdrive/My Drive/music/rock.zip"
!unzip "/content/gdrive/My Drive/music/pop.zip"

Archive:  /content/gdrive/My Drive/music/pop.zip
   creating: pop/
  inflating: pop/100.mid             
  inflating: pop/1000.mid            
  inflating: pop/1001.mid            
  inflating: pop/1002.mid            
  inflating: pop/1003.mid            
  inflating: pop/1004.mid            
  inflating: pop/1005.mid            
  inflating: pop/1006.mid            
  inflating: pop/1007.mid            
  inflating: pop/1008.mid            
  inflating: pop/1009.mid            
  inflating: pop/101.mid             
  inflating: pop/1010.mid            
  inflating: pop/1011.mid            
  inflating: pop/1012.mid            
  inflating: pop/1013.mid            
  inflating: pop/1014.mid            
  inflating: pop/1015.mid            
  inflating: pop/1016.mid            
  inflating: pop/1017.mid            
  inflating: pop/1018.mid            
  inflating: pop/1019.mid            
  inflating: pop/102.mid             
  inflating: pop/1020.mid            
  inflating: pop/1021

In [0]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

%tensorflow_version 1.x

Using TensorFlow backend.


In [0]:
dir='pop'

In [0]:
from keras.models import load_model
from os import path

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob(dir+"/*.mid")[:30]:
        try:
        
          midi = converter.parse(file)

          print("Parsing %s" % file)

          notes_to_parse = None

          try: # file has instrument parts
              s2 = instrument.partitionByInstrument(midi)
              notes_to_parse = s2.parts[0].recurse() 
          except: # file has notes in a flat structure
              notes_to_parse = midi.flat.notes

          for element in notes_to_parse:
              if isinstance(element, note.Note):
                  notes.append(str(element.pitch))
              elif isinstance(element, chord.Chord):
                  notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
          print(e)


    with open('notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 50

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    '''
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    '''
    #print((network_input.shape[1], network_input.shape[2])," ",n_vocab)


    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (network_input.shape[1], network_input.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    if path.exists('./'+dir+'_300_model.h5'):
      model = load_model('./'+dir+'_300_model.h5')
    
    #print(network_input.shape , " ", network_output.shape)
    model.fit(network_input, network_output, epochs=200, batch_size=128) #, callbacks=callbacks_list)

    model.save(dir+'_300_model.h5')

In [0]:
train_network()

Parsing pop/239.mid
Parsing pop/891.mid
Parsing pop/139.mid
Parsing pop/611.mid
Parsing pop/917.mid
Parsing pop/382.mid
Parsing pop/4475.mid
Parsing pop/932.mid
Parsing pop/351.mid
Parsing pop/909.mid
Parsing pop/242.mid
Parsing pop/300.mid
Parsing pop/450.mid
Parsing pop/971.mid
Parsing pop/230.mid
Parsing pop/1000.mid
Parsing pop/610.mid
Parsing pop/748.mid
Parsing pop/115.mid
Parsing pop/812.mid
Parsing pop/373.mid
Parsing pop/478.mid
Parsing pop/279.mid
Parsing pop/722.mid
Parsing pop/130.mid
Parsing pop/694.mid
Parsing pop/545.mid
Parsing pop/34.mid
Parsing pop/753.mid
Parsing pop/692.mid
Epoch 1/200
30374/30374 [==============================] - 76s 2ms/step - loss: 1.5905 - acc: 0.5471
Epoch 2/200
30374/30374 [==============================] - 75s 2ms/step - loss: 1.5851 - acc: 0.5483
Epoch 3/200
30374/30374 [==============================] - 75s 2ms/step - loss: 1.5797 - acc: 0.5503
Epoch 4/200
30374/30374 [==============================] - 75s 2ms/step - loss: 1.5863 - acc: 0.

In [0]:

import pickle
import numpy
from music21 import instrument, note, stream, chord
import time



def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences1(notes, pitchnames, n_vocab)
    model = create_network1(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences1(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 50
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network1(network_input, n_vocab):
    """ create the structure of the neural network """
    '''
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('weights.hdf5')
    '''
    model = load_model(dir+'_model.h5')
    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    #atime = time.time()

    start = numpy.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []

    #ct=0
    #btime = time.time()
    # generate 500 notes
    for note_index in range(200):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        #atime = time.time()
        prediction = model.predict(prediction_input, verbose=0)
        #ct+= time.time() - atime

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    #print((time.time() - btime) - ct)

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')


In [0]:
generate()

0.02004861831665039
